In [1]:
import tkinter as tk
from tkinter import ttk, messagebox
from datetime import datetime
import pandas as pd
import os

app = tk.Tk()
app.title("MindEase: Wellness Tracker")
app.geometry("1080x720")
app.configure(bg="#f5f7fa")

# Theme colors
bg_color = "#f5f7fa"
frame_color = "#ffffff"
text_color = "#2d3436"
accent_color = "#0984e3"
error_color = "#d63031"
secondary_color = "#00b894"

name_input = tk.StringVar()
wellness_input = tk.StringVar()
metime_input = tk.StringVar()
screen_time_input = tk.StringVar()
frequency_input = tk.StringVar()
today_date = tk.StringVar(value=datetime.today().strftime('%Y-%m-%d'))

frequency_list = ["Daily", "2-3 times/week", "Once a week", "Rarely"]

def check_status(minutes):
    return "Healthy" if minutes >= 60 else "Needs Improvement"

def status_color(status_text):
    return {"Healthy": secondary_color, "Needs Improvement": error_color}.get(status_text, text_color)

def fade_in(window):
    alpha = 0.0
    step = 0.05
    def increase():
        nonlocal alpha
        if alpha < 1:
            alpha += step
            window.attributes('-alpha', alpha)
            window.after(30, increase)
    increase()

def open_input_window():
    input_win = tk.Toplevel(app)
    input_win.title("New Entry Form")
    input_win.geometry("500x420")
    input_win.configure(bg=bg_color)
    fade_in(input_win)

    input_frame = tk.Frame(input_win, padx=15, pady=15, bg=frame_color)
    input_frame.pack(fill="both", expand=True, padx=20, pady=20)

    def create_label_input(row_no, label_txt, variable, input_type="entry", choices=None):
        tk.Label(input_frame, text=label_txt, font=("Arial", 12), bg=frame_color, fg=text_color).grid(row=row_no, column=0, sticky="w", padx=10, pady=7)
        if input_type == "entry":
            entry = tk.Entry(input_frame, textvariable=variable, font=("Arial", 12), bg="#f8f8f8")
        elif input_type == "combo":
            entry = ttk.Combobox(input_frame, textvariable=variable, values=choices, state="readonly", font=("Arial", 12))
            entry.current(0)
        entry.grid(row=row_no, column=1, sticky="ew", padx=10, pady=7)

    input_frame.columnconfigure(1, weight=1)
    create_label_input(0, "Student Name", name_input)
    create_label_input(1, "Wellness Task", wellness_input)
    create_label_input(2, "Relaxing Activity", metime_input)
    create_label_input(3, "Offline Time (minutes)", screen_time_input)
    create_label_input(4, "How Often?", frequency_input, "combo", frequency_list)

    def submit_from_window():
        add_record()
        input_win.destroy()

    tk.Button(input_frame, text="Add This Entry", command=submit_from_window, bg=accent_color, fg="white", font=("Arial", 11)).grid(row=5, column=0, columnspan=2, pady=10)

def open_summary_window():
    summary_win = tk.Toplevel(app)
    summary_win.title("Activity Overview")
    summary_win.geometry("400x300")
    summary_win.configure(bg=bg_color)
    fade_in(summary_win)

    summary_frame = tk.Frame(summary_win, padx=15, pady=15, bg=frame_color)
    summary_frame.pack(fill="both", expand=True, padx=20, pady=20)

    total_entries = len(record_table.get_children())
    healthy_count = sum(1 for row in record_table.get_children() if record_table.item(row)['values'][-1] == "Healthy")
    me_time_sum = sum(int(record_table.item(row)['values'][4]) for row in record_table.get_children())

    tk.Label(summary_frame, text=f"Entries Logged: {total_entries}", font=("Arial", 12), bg=frame_color, fg=text_color).pack(pady=5)
    tk.Label(summary_frame, text=f"Healthy Days: {healthy_count}", font=("Arial", 12), bg=frame_color, fg=secondary_color).pack(pady=5)
    tk.Label(summary_frame, text=f"Total Relaxation Time: {me_time_sum} mins", font=("Arial", 12), bg=frame_color, fg=accent_color).pack(pady=5)

def open_help():
    messagebox.showinfo("How to Use", "\n1. Use 'Add New Entry' to log your wellness data.\n2. View summary via 'Activity Overview'.\n3. Use 'Export Data' to save your logs.")

def open_about():
    messagebox.showinfo("About MindEase", "MindEase helps students track their mental wellness and offline activities.\n\nBuilt with Python and Tkinter.")

menu_bar = tk.Menu(app)
app.config(menu=menu_bar)

entry_menu = tk.Menu(menu_bar, tearoff=0)
entry_menu.add_command(label="Activity Overview", command=open_summary_window)
entry_menu.add_separator()
entry_menu.add_command(label="Exit App", command=app.quit)

help_menu = tk.Menu(menu_bar, tearoff=0)
help_menu.add_command(label="How to Use", command=open_help)
help_menu.add_command(label="About MindEase", command=open_about)

menu_bar.add_cascade(label="Menu", menu=entry_menu)
menu_bar.add_cascade(label="Help", menu=help_menu)

input_box = tk.LabelFrame(app, text="Quick Entry", font=("Arial", 14, "bold"), bg=frame_color, fg=text_color, padx=15, pady=15)
input_box.pack(fill="x", padx=20, pady=15)

def create_label_input(row_no, label_txt, variable, input_type="entry", choices=None):
    tk.Label(input_box, text=label_txt, font=("Arial", 12), bg=frame_color, fg=text_color).grid(row=row_no, column=0, sticky="w", padx=10, pady=7)
    if input_type == "entry":
        entry = tk.Entry(input_box, textvariable=variable, font=("Arial", 12), bg="#f8f8f8")
    elif input_type == "combo":
        entry = ttk.Combobox(input_box, textvariable=variable, values=choices, state="readonly", font=("Arial", 12))
        entry.current(0)
    entry.grid(row=row_no, column=1, sticky="ew", padx=10, pady=7)

input_box.columnconfigure(1, weight=1)
create_label_input(0, "Student Name", name_input)
create_label_input(1, "Wellness Task", wellness_input)
create_label_input(2, "Relaxing Activity", metime_input)
create_label_input(3, "Offline Time (minutes)", screen_time_input)
create_label_input(4, "How Often?", frequency_input, "combo", frequency_list)

status_display = tk.Label(input_box, text="", font=("Arial", 12, "bold"), bg=frame_color, fg=text_color)
status_display.grid(row=5, column=0, columnspan=2, pady=10)

buttons_frame = tk.Frame(input_box, bg=frame_color)
buttons_frame.grid(row=6, column=0, columnspan=2, pady=15)

def reset_fields():
    name_input.set("")
    wellness_input.set("")
    metime_input.set("")
    screen_time_input.set("")
    frequency_input.set(frequency_list[0])
    status_display.config(text="")

def add_record():
    try:
        student = name_input.get().strip()
        wellness = wellness_input.get().strip()
        metime = metime_input.get().strip()
        screen = screen_time_input.get().strip()
        freq = frequency_input.get().strip()

        if not student or not wellness or not metime or not screen or not freq:
            raise ValueError("All fields are required.")

        if not screen.isdigit() or int(screen) < 0:
            raise ValueError("Screen-Free Time must be a positive number.")

        screen_val = int(screen)
        status_now = check_status(screen_val)

        timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        record_table.insert("", "end", values=(timestamp, student, wellness, metime, screen_val, freq, status_now))
        status_display.config(text=f"Status: {status_now}", fg=status_color(status_now))
        messagebox.showinfo("Success", "Entry added successfully.")
        reset_fields()

    except ValueError as e:
        messagebox.showerror("Error", str(e))

def remove_selected():
    selected = record_table.selection()
    if not selected:
        messagebox.showerror("Error", "No entry selected.")
    else:
        record_table.delete(selected[0])
        messagebox.showinfo("Deleted", "Selected entry removed.")

def clear_everything():
    for row in record_table.get_children():
        record_table.delete(row)
    messagebox.showinfo("Cleared", "All entries cleared.")

def export_to_excel():
    records = [record_table.item(row)["values"] for row in record_table.get_children()]
    if not records:
        messagebox.showwarning("Empty", "No entries to save.")
        return
    df = pd.DataFrame(records, columns=["Timestamp", "Name", "Wellness Activity", "Me-Time Activity", "Offline Time (min)", "Frequency", "Status"])
    filename = "mind_ease_log.xlsx"
    try:
        if os.path.exists(filename):
            df_existing = pd.read_excel(filename)
            df_combined = pd.concat([df_existing, df], ignore_index=True)
            df_combined.to_excel(filename, index=False)
        else:
            df.to_excel(filename, index=False)
        messagebox.showinfo("Saved", f"Data saved to {filename}")
    except Exception as e:
        messagebox.showerror("Error", f"Save failed: {e}")

tk.Button(buttons_frame, text="Add Entry", command=add_record, bg=accent_color, fg="white", width=15, font=("Arial", 11)).grid(row=0, column=0, padx=8)
tk.Button(buttons_frame, text="Delete Entry", command=remove_selected, bg=error_color, fg="white", width=15, font=("Arial", 11)).grid(row=0, column=1, padx=8)
tk.Button(buttons_frame, text="Clear All", command=clear_everything, bg="#636e72", fg="white", width=15, font=("Arial", 11)).grid(row=0, column=2, padx=8)
tk.Button(buttons_frame, text="Export Data", command=export_to_excel, bg=secondary_color, fg="white", width=18, font=("Arial", 11)).grid(row=0, column=3, padx=8)

table_section = tk.LabelFrame(app, text="Wellness History", font=("Arial", 14, "bold"), bg=frame_color, fg=text_color)
table_section.pack(fill='both', expand=True, padx=20, pady=15)

column_names = ("timestamp", "name", "wellness", "metime", "screen", "frequency", "status")
record_table = ttk.Treeview(table_section, columns=column_names, show="headings", height=12)

for col in column_names:
    record_table.heading(col, text=col.replace("_", " ").title())
    record_table.column(col, width=130, anchor="center")

record_table.pack(fill="both", expand=True)

scroll = ttk.Scrollbar(table_section, orient="vertical", command=record_table.yview)
record_table.configure(yscrollcommand=scroll.set)
scroll.pack(side="right", fill="y")

def reminder_popup():
    messagebox.showinfo("Reminder", "Take a moment to log how you're feeling today!")
    app.after(8 * 60 * 60 * 1000, reminder_popup)

app.after(2000, reminder_popup)
app.mainloop()
